# Set Up

In [ ]:
import sys
import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
import torch.utils.data as Data
import csv
import scipy.io
import matplotlib.pyplot as plt
import scipy.stats
import random
import warnings
import glob
import matplotlib.ticker as mticker
from sklearn.model_selection import KFold, cross_val_score
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import interp1d
from scipy.stats import norm
from matplotlib.lines import Line2D
from tqdm import tqdm
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import accuracy_score
from matplotlib.pyplot import figure
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LinearRegression
from mlxtend.evaluate import ttest
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.model_selection import GridSearchCV
from sklearn.utils import resample
from sklearn.metrics import c_score
from matplotlib.ticker import PercentFormatter
from lifelines import KaplanMeierFitter
from lifelines.utils import qth_survival_times
from lifelines.statistics import logrank_test
from lifelines import KaplanMeierFitter
from sklearn.calibration import calibration_curve
from matplotlib import rcParams

In [ ]:
def compute_midrank(x):
   J = np.argsort(x)
   Z = x[J]
   N = len(x)
   T = np.zeros(N, dtype=np.float64)
   i = 0
   while i < N:
       j = i
       while j < N and Z[j] == Z[i]:
           j += 1
       T[i:j] = 0.5*(i + j - 1)
       i = j
   T2 = np.empty(N, dtype=np.float64)
   T2[J] = T + 1
   return T2

def compute_midrank_weight(x, sample_weight):
   J = np.argsort(x)
   Z = x[J]
   cumulative_weight = np.cumsum(sample_weight[J])
   N = len(x)
   T = np.zeros(N, dtype=np.float64)
   i = 0
   while i < N:
       j = i
       while j < N and Z[j] == Z[i]:
           j += 1
       T[i:j] = cumulative_weight[i:j].mean()
       i = j
   T2 = np.empty(N, dtype=np.float64)
   T2[J] = T
   return T2

def fastDeLong(predictions_sorted_transposed, label_1_count, sample_weight=None):
   if sample_weight is None:
       return fastDeLong_no_weights(predictions_sorted_transposed, label_1_count)
   else:
       return fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight)

def fastDeLong_weights(predictions_sorted_transposed, label_1_count, sample_weight):
   m = label_1_count
   n = predictions_sorted_transposed.shape[1] - m
   positive_examples = predictions_sorted_transposed[:, :m]
   negative_examples = predictions_sorted_transposed[:, m:]
   k = predictions_sorted_transposed.shape[0]
   tx = np.empty([k, m], dtype=np.float64)
   ty = np.empty([k, n], dtype=np.float64)
   tz = np.empty([k, m + n], dtype=np.float64)
   for r in range(k):
       tx[r, :] = compute_midrank_weight(positive_examples[r, :], sample_weight[:m])
       ty[r, :] = compute_midrank_weight(negative_examples[r, :], sample_weight[m:])
       tz[r, :] = compute_midrank_weight(predictions_sorted_transposed[r, :], sample_weight)
   total_positive_weights = sample_weight[:m].sum()
   total_negative_weights = sample_weight[m:].sum()
   pair_weights = np.dot(sample_weight[:m, np.newaxis], sample_weight[np.newaxis, m:])
   total_pair_weights = pair_weights.sum()
   aucs = (sample_weight[:m]*(tz[:, :m] - tx)).sum(axis=1) / total_pair_weights
   v01 = (tz[:, :m] - tx[:, :]) / total_negative_weights
   v10 = 1. - (tz[:, m:] - ty[:, :]) / total_positive_weights
   sx = np.cov(v01)
   sy = np.cov(v10)
   delongcov = sx / m + sy / n
   return aucs, delongcov

def fastDeLong_no_weights(predictions_sorted_transposed, label_1_count):
   m = label_1_count
   n = predictions_sorted_transposed.shape[1] - m
   positive_examples = predictions_sorted_transposed[:, :m]
   negative_examples = predictions_sorted_transposed[:, m:]
   k = predictions_sorted_transposed.shape[0]
   tx = np.empty([k, m], dtype=np.float64)
   ty = np.empty([k, n], dtype=np.float64)
   tz = np.empty([k, m + n], dtype=np.float64)
   for r in range(k):
       tx[r, :] = compute_midrank(positive_examples[r, :])
       ty[r, :] = compute_midrank(negative_examples[r, :])
       tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
   aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
   v01 = (tz[:, :m] - tx[:, :]) / n
   v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
   sx = np.cov(v01)
   sy = np.cov(v10)
   delongcov = sx / m + sy / n
   return aucs, delongcov

def calc_pvalue(aucs, sigma):
   l = np.array([[1, -1]])
   z = np.abs(np.diff(aucs)) / (np.sqrt(np.dot(np.dot(l, sigma), l.T)) + 1e-8)
   pvalue = 2 * (1 - scipy.stats.norm.cdf(np.abs(z)))
   return pvalue

def compute_ground_truth_statistics(ground_truth, sample_weight=None):
   assert np.array_equal(np.unique(ground_truth), [0, 1])
   order = (-ground_truth).argsort()
   label_1_count = int(ground_truth.sum())
   if sample_weight is None:
       ordered_sample_weight = None
   else:
       ordered_sample_weight = sample_weight[order]
   return order, label_1_count, ordered_sample_weight

def delong_roc_variance(ground_truth, predictions):
   sample_weight = None
   order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
       ground_truth, sample_weight)
   predictions_sorted_transposed = predictions[np.newaxis, order]
   aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
   assert len(aucs) == 1,
   return aucs[0], delongcov

def delong_roc_test(ground_truth, predictions_one, predictions_two):
   sample_weight = None
   order, label_1_count,ordered_sample_weight = compute_ground_truth_statistics(ground_truth)
   predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
   aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count,sample_weight)
   return calc_pvalue(aucs, delongcov)

def delong_roc_ci(y_true,y_pred):
   aucs, auc_cov = delong_roc_variance(y_true, y_pred)
   auc_std = np.sqrt(auc_cov)
   lower_upper_q = np.abs(np.array([0, 1]) - (1 - 0.95) / 2) # alpha=0.95
   ci = stats.norm.ppf(
       lower_upper_q,
       loc=aucs,
       scale=auc_std)
   ci[ci > 1] = 1
   return aucs,ci

# Main analysis

In [ ]:
path0='/UKB协变量_ysp20230503.dta'
path1='/UKB代谢组_ysp20230429.dta'
path2='/UKB心血管结局_ysp20230429.dta'
path3='/UKB_dmstatus.dta'

convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1', 'a2', 'a6',  'a8', 'a13','a16','chol_mmol','hdl']]
dm = dm[['eid_ageing','dmstatus']]

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

In [ ]:
event = event[['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']]
c_table = pd.DataFrame()
color_map=['#C6B69D','#A19275','#949496','#DFDCD7','#828B78','#7C785D','#D4CBAC','#EBDBA7','#B27F9E','#B1764C']
conv=['a1', 'a2', 'a6',  'a8', 'a13','a16','chol_mmol','hdl', 'dmstatus','rnfl']

for i in tqdm(range(event.shape[1]-1)):

  plt.figure(figsize=(5,5))
  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(convar,dm, on='eid_ageing')
  merged_df = pd.merge(merged_df,metab, on='eid_ageing')
  merged_df = pd.merge(merged_df,disease, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  train_ratio=0.8
  test_ratio=0.2
  num_train_samples = int(len(df_shuffled) * train_ratio)
  num_test_samples = len(df_shuffled) - num_train_samples
  train_data = df_shuffled[:num_train_samples].reset_index(drop=True)
  test_data = df_shuffled[num_train_samples:].reset_index(drop=True)
  train_data.drop(['eid_ageing'], axis=1, inplace=True)
  test_data.drop(['eid_ageing'], axis=1, inplace=True)

  for j in range(10):

    x_train_1 = train_data.iloc[:, j]
    y_train_1 = train_data.iloc[:, -1]
    x_test_1 = test_data.iloc[:, j]
    y_test_1 = test_data.iloc[:, -1]
    xtrain1 = x_train_1.to_numpy()
    ytrain1 = y_train_1.to_numpy()
    xtest1 = x_test_1.to_numpy()
    ytest1 = y_test_1.to_numpy()

    rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_1.fit(xtrain1, ytrain1)
    ypred1 = rf_1.predict(xtest1)
    fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
    c1 = auc(fpr1, tpr1)

    # Results
    c_table.loc[i, j] = c1
    col_name = disease.columns[1]
    arr_path = ('/results/auc/ypred1_'+f"{col_name}" +'.npy')
    np.save(arr_path, ypred1)
    color = color_map[j]
    label = conv[j]+' (area = %0.3f)' % c1
    plt.plot(fpr1, tpr1, color=color, lw=1, label=label)

  plt.plot([0, 1], [0, 1], color='#E1DCD9', lw=1, linestyle='--')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('roc_curve_' +f"{col_name}")
  plt.legend(loc="lower right")
  plt.savefig('/results/roc_curve_' +f"{col_name}" + '.png')
  plt.clf()

  arr_path = ('ytest_'+f"{col_name}" + '.npy')
  np.save(arr_path, ytest1)

c_table.to_csv('c_table.csv', index=False)

## Models Comparison (Simple Model)

In [ ]:
# Load data
path0='/UKB协变量_ysp20230503.dta'
path1='/UKB代谢组_ysp20230429.dta'
path2='/UKB心血管结局_ysp20230429.dta'
path3='/UKB_dmstatus.dta'

convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1','a2']]

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

In [ ]:
event = event[['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']]
c_table = pd.DataFrame(columns=['RF_CB', 'RF_SB', 'RF_RNFL', 'P-value'])

for i in tqdm(range(event.shape[1]-1)):

  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  train_ratio=0.8
  test_ratio=0.2
  num_train_samples = int(len(df_shuffled) * train_ratio)
  num_test_samples = len(df_shuffled) - num_train_samples
  train_data = df_shuffled[:num_train_samples].reset_index(drop=True)
  test_data = df_shuffled[num_train_samples:].reset_index(drop=True)
  train_data.drop(['eid_ageing'], axis=1, inplace=True)
  test_data.drop(['eid_ageing'], axis=1, inplace=True)

  x_train_1 = train_data.iloc[:, :-1]
  y_train_1 = train_data.iloc[:, -1]
  x_test_1 = test_data.iloc[:, :-1]
  y_test_1 = test_data.iloc[:, -1]
  xtrain1 = x_train_1.to_numpy()
  ytrain1 = y_train_1.to_numpy()
  xtest1 = x_test_1.to_numpy()
  ytest1 = y_test_1.to_numpy()
  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)
  fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
  c1 = auc(fpr1, tpr1)

  train_data_2 = train_data.iloc[:, :2]
  train_label_2 = train_data.iloc[:, -1]
  test_data_2 = test_data.iloc[:, :2]
  test_label_2 = test_data.iloc[:, -1]
  train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
  test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)
  x_train_2 = train_data_2.iloc[:, :-1]
  y_train_2 = train_data_2.iloc[:, -1]
  x_test_2 = test_data_2.iloc[:, :-1]
  y_test_2 = test_data_2.iloc[:, -1]
  xtrain2 = x_train_2.to_numpy()
  ytrain2 = y_train_2.to_numpy()
  xtest2 = x_test_2.to_numpy()
  ytest2 = y_test_2.to_numpy()
  rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_2.fit(xtrain2, ytrain2)
  ypred2 = rf_2.predict(xtest2)
  fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
  c2 = auc(fpr2, tpr2)

  x_train_3 = train_data.iloc[:, 2:-1]
  y_train_3 = train_data.iloc[:, -1]
  x_test_3 = test_data.iloc[:, 2:-1]
  y_test_3 = test_data.iloc[:, -1]
  xtrain3 = x_train_3.to_numpy()
  ytrain3 = y_train_3.to_numpy()
  xtest3 = x_test_3.to_numpy()
  ytest3 = y_test_3.to_numpy()
  rf_3 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_3.fit(xtrain3, ytrain3)
  ypred3 = rf_3.predict(xtest3)
  fpr3, tpr3, _ = roc_curve(ytest3, ypred3)
  c3 = auc(fpr3, tpr3)

  p = delong_roc_test(ytest1,ypred1,ypred2)
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)
  aus2,ci2 = delong_roc_ci(ytest1,ypred2)

  c_table.loc[len(c_table)] = [c1, c2, c3, p]
  col_name = disease.columns[1]


  arr_path = ('ypred1_'+f"{col_name}" +'.npy')
  np.save(arr_path, ypred1)
  arr_path = ('ypred2_'+f"{col_name}" +'.npy')
  np.save(arr_path, ypred2)
  arr_path = ('ypred3_'+f"{col_name}" +'.npy')
  np.save(arr_path, ypred3)
  arr_path = ('ytest_'+f"{col_name}" +'.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

## Models Comparison (FGCRS)

In [ ]:
# Load data
path0='/UKB协变量_ysp20230503.dta'
path1='/UKB代谢组_ysp20230429.dta'
path2='/UKB心血管结局_ysp20230429.dta'
path3='/UKB_dmstatus.dta'

convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1','a2','a6','a16','a13','chol_mmol','hdl']]
dm = dm[['eid_ageing','dmstatus']]

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1','a16','chol_mmol','hdl']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

In [ ]:
event = event[['eid_ageing','t2d_2104','mi_2104','cvddeath_status','cvd_hf_2104','cerestroke_2104','deathstatus']]
c_table = pd.DataFrame(columns=['CB_CIL', 'CB_ROC', 'CB_CIH', 'CV_CIL', 'CV_ROC', 'CV_CIH', 'P'])

for i in tqdm(range(event.shape[1]-1)):

  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  train_ratio=0.8
  num_train_samples = int(len(df_shuffled) * train_ratio)
  num_test_samples = len(df_shuffled) - num_train_samples
  train_data = df_shuffled[:num_train_samples].reset_index(drop=True)
  test_data = df_shuffled[num_train_samples:].reset_index(drop=True)
  train_data.drop(['eid_ageing'], axis=1, inplace=True)
  test_data.drop(['eid_ageing'], axis=1, inplace=True)

  # Combined Model
  age_idx = train_data.columns.get_loc('a1')
  x_train_1 = train_data.iloc[:, age_idx:-1]
  y_train_1 = train_data.iloc[:, -1]
  x_test_1 = test_data.iloc[:, age_idx:-1]
  y_test_1 = test_data.iloc[:, -1]
  xtrain1 = x_train_1.to_numpy()
  ytrain1 = y_train_1.to_numpy()
  xtest1 = x_test_1.to_numpy()
  ytest1 = y_test_1.to_numpy()


  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)
  fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
  c1 = auc(fpr1, tpr1)

  # FGCRS
  metab_idx = train_data.columns.get_loc('m7')
  train_data_2 = train_data.iloc[:, age_idx:metab_idx]
  train_label_2 = train_data.iloc[:, -1]
  test_data_2 = test_data.iloc[:, age_idx:metab_idx]
  test_label_2 = test_data.iloc[:, -1]
  train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
  test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

  x_train_2 = train_data_2.iloc[:, :-1]
  y_train_2 = train_data_2.iloc[:, -1]
  x_test_2 = test_data_2.iloc[:, :-1]
  y_test_2 = test_data_2.iloc[:, -1]
  xtrain2 = x_train_2.to_numpy()
  ytrain2 = y_train_2.to_numpy()
  xtest2 = x_test_2.to_numpy()
  ytest2 = y_test_2.to_numpy()

  rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_2.fit(xtrain2, ytrain2)
  ypred2 = rf_2.predict(xtest2)
  fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
  c2 = auc(fpr2, tpr2)


  #Delong Test
  p = delong_roc_test(ytest1,ypred1,ypred2)
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)
  aus2,ci2 = delong_roc_ci(ytest1,ypred2)

  # Results
  c_table.loc[len(c_table)] = [ci1[0], aus1, ci1[1], ci2[0], aus2, ci2[1], p[0,0]]
  col_name = disease.columns[1]



  arr_path = ('ypred1_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ypred1)
  arr_path = ('ypred2_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ypred2)
  arr_path = ('ytest_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)


## FGCRS T2D Model (Supplementary)

In [ ]:
# Load data
path0='UKB协变量T2D_ysp20230718.dta'
path1='UKB心血管结局_ysp20230429.dta'
path2='UKB代谢组_ysp20230429.dta'
convar=pd.read_stata(path0)
event=pd.read_stata(path1)
metab=pd.read_stata(path2)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]

In [ ]:
scaler = MinMaxScaler()
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

# Build conventional model
concise = convar[['eid_ageing','baselineage','sex','family_diabetes','bmi','hba1c']]
full = convar[['eid_ageing','baselineage','sex','family_diabetes','bmi','hba1c','hbp','hdl','tg','waist']]

### Model Comparison

In [ ]:
# Model Training
event = event[['eid_ageing','t2d_2104']]
models = ['concise','full']
c_table = pd.DataFrame(columns=['RF_CB', 'RF_FGCRS', 'P-value'])

for i in tqdm(range(2)):

  model_name = models[i]

  if i==0:  #concise model
    merged_df = pd.merge(metab,event, on='eid_ageing')
    merged_df = pd.merge(concise,merged_df, on='eid_ageing')
    merged_df = merged_df.dropna()
    df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  if i==1:  #full model
    merged_df = pd.merge(metab,event, on='eid_ageing')
    merged_df = pd.merge(full,merged_df, on='eid_ageing')
    merged_df = merged_df.dropna()
    df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  train_ratio=0.8
  test_ratio=0.2
  num_train_samples = int(len(df_shuffled) * train_ratio)
  num_test_samples = len(df_shuffled) - num_train_samples
  train_data = df_shuffled[:num_train_samples].reset_index(drop=True)
  test_data = df_shuffled[num_train_samples:].reset_index(drop=True)
  train_data.drop(['eid_ageing'], axis=1, inplace=True)
  test_data.drop(['eid_ageing'], axis=1, inplace=True)

  # Combined Model
  x_train_1 = train_data.iloc[:, :-1]
  y_train_1 = train_data.iloc[:, -1]
  x_test_1 = test_data.iloc[:, :-1]
  y_test_1 = test_data.iloc[:, -1]
  xtrain1 = x_train_1.to_numpy()
  ytrain1 = y_train_1.to_numpy()
  xtest1 = x_test_1.to_numpy()
  ytest1 = y_test_1.to_numpy()

  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)
  fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
  c1 = auc(fpr1, tpr1)

  # FGCRS
  train_data_2 = train_data.iloc[:, :-27]
  train_label_2 = train_data.iloc[:, -1]
  test_data_2 = test_data.iloc[:, :-27]
  test_label_2 = test_data.iloc[:, -1]
  train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
  test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

  x_train_2 = train_data_2.iloc[:, :-1]
  y_train_2 = train_data_2.iloc[:, -1]
  x_test_2 = test_data_2.iloc[:, :-1]
  y_test_2 = test_data_2.iloc[:, -1]
  xtrain2 = x_train_2.to_numpy()
  ytrain2 = y_train_2.to_numpy()
  xtest2 = x_test_2.to_numpy()
  ytest2 = y_test_2.to_numpy()

  rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_2.fit(xtrain2, ytrain2)
  ypred2 = rf_2.predict(xtest2)
  fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
  c2 = auc(fpr2, tpr2)

  #Delong Test
  p = delong_roc_test(ytest1,ypred1,ypred2)
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)
  aus2,ci2 = delong_roc_ci(ytest1,ypred2)

  # Results
  c_table.loc[len(c_table)] = [c1, c2, p]


  arr_path = ('ypred1_t2d_'+f"{model_name}"+ '.npy')
  np.save(arr_path, ypred1)
  arr_path = ('ypred2_t2d_'+f"{model_name}"+ '.npy')
  np.save(arr_path, ypred2)
  arr_path = ('ytest_t2d_'+f"{model_name}"+ '.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

### Stratified Study  (Sex)

In [ ]:
event = event[['eid_ageing','t2d_2104']]
c_table = pd.DataFrame(columns=['RF_CB', 'RF_FGCRS', 'P-value'])

# if need to balance dataset
balance = 0

merged_df = pd.merge(metab,event, on='eid_ageing')
merged_df = pd.merge(full,merged_df, on='eid_ageing')
merged_df = merged_df.dropna()
df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

for j in tqdm(range(2)):
  sex = df_shuffled.loc[df_shuffled['sex'] == j]
  sex.drop(['sex'], axis=1, inplace=True)

  train_ratio=0.8
  test_ratio=0.2
  num_train_samples = int(len(sex) * train_ratio)
  num_test_samples = len(sex) - num_train_samples
  train_data = sex[:num_train_samples].reset_index(drop=True)
  test_data = sex[num_train_samples:].reset_index(drop=True)
  train_data.drop(['eid_ageing'], axis=1, inplace=True)
  test_data.drop(['eid_ageing'], axis=1, inplace=True)

  if balance == 1:

    female = len(df_shuffled.loc[df_shuffled['sex'] == 0])
    male = len(df_shuffled.loc[df_shuffled['sex'] == 1])

    if male > female and j==0:
      indexlist = list(range(1,int(female*0.8)))
      random.seed(100)
      oversample = np.array(random.sample(indexlist, k=int((male-female)*0.8)))
      aug = train_data.iloc[oversample, :]
      train_data = pd.concat([train_data, aug], axis=0)

    if female >= male and j==1:
      indexlist = list(range(1,int(male*0.8)))
      random.seed(100)
      oversample = np.array(random.sample(indexlist, k=int((female-male)*0.8)))
      aug = train_data.iloc[oversample, :]
      train_data = pd.concat([train_data, aug], axis=0)

  # Combined Model
  x_train_1 = train_data.iloc[:, :-1]
  y_train_1 = train_data.iloc[:, -1]
  x_test_1 = test_data.iloc[:, :-1]
  y_test_1 = test_data.iloc[:, -1]
  xtrain1 = x_train_1.to_numpy()
  ytrain1 = y_train_1.to_numpy()
  xtest1 = x_test_1.to_numpy()
  ytest1 = y_test_1.to_numpy()

  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)
  fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
  c1 = auc(fpr1, tpr1)

  # FGCRS
  train_data_2 = train_data.iloc[:, :-27]
  train_label_2 = train_data.iloc[:, -1]
  test_data_2 = test_data.iloc[:, :-27]
  test_label_2 = test_data.iloc[:, -1]
  train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
  test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

  x_train_2 = train_data_2.iloc[:, :-1]
  y_train_2 = train_data_2.iloc[:, -1]
  x_test_2 = test_data_2.iloc[:, :-1]
  y_test_2 = test_data_2.iloc[:, -1]
  xtrain2 = x_train_2.to_numpy()
  ytrain2 = y_train_2.to_numpy()
  xtest2 = x_test_2.to_numpy()
  ytest2 = y_test_2.to_numpy()

  rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_2.fit(xtrain2, ytrain2)
  ypred2 = rf_2.predict(xtest2)
  fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
  c2 = auc(fpr2, tpr2)

  #Delong Test
  p = delong_roc_test(ytest1,ypred1,ypred2)
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)
  aus2,ci2 = delong_roc_ci(ytest1,ypred2)

  c_table.loc[len(c_table)] = [c1, c2, p]



  arr_path = ('ypred1_t2d_'+ str(j)+ '.npy')
  np.save(arr_path, ypred1)
  arr_path = ('auc/ypred2_t2d_'+ str(j)+ '.npy')
  np.save(arr_path, ypred2)
  arr_path = ('ytest_t2d_'+ str(j) + '.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

### Stratified Study  (Edu)

In [ ]:
path3='UKB协变量_ysp20230503.dta'
convar2 = pd.read_stata(path3)
convar2 = convar2[['eid_ageing','a9']]

In [ ]:
event = event[['eid_ageing','t2d_2104']]
c_table = pd.DataFrame(columns=['RF_CB', 'RF_FGCRS', 'P-value'])

merged_df = pd.merge(metab,event, on='eid_ageing')
merged_df = pd.merge(full,merged_df, on='eid_ageing')
merged_df = pd.merge(convar2,merged_df, on='eid_ageing')
merged_df = merged_df.dropna()
df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

for j in tqdm(range(2)):
  edu = df_shuffled.loc[df_shuffled['a9'] == j]
  edu.drop(['a9'], axis=1, inplace=True)

  train_ratio=0.8
  test_ratio=0.2
  num_train_samples = int(len(edu) * train_ratio)
  num_test_samples = len(edu) - num_train_samples
  train_data = edu[:num_train_samples].reset_index(drop=True)
  test_data = edu[num_train_samples:].reset_index(drop=True)
  train_data.drop(['eid_ageing'], axis=1, inplace=True)
  test_data.drop(['eid_ageing'], axis=1, inplace=True)

  # Combined Model
  x_train_1 = train_data.iloc[:, :-1]
  y_train_1 = train_data.iloc[:, -1]
  x_test_1 = test_data.iloc[:, :-1]
  y_test_1 = test_data.iloc[:, -1]
  xtrain1 = x_train_1.to_numpy()
  ytrain1 = y_train_1.to_numpy()
  xtest1 = x_test_1.to_numpy()
  ytest1 = y_test_1.to_numpy()

  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)
  fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
  c1 = auc(fpr1, tpr1)

  # FGCRS
  train_data_2 = train_data.iloc[:, :-27]
  train_label_2 = train_data.iloc[:, -1]
  test_data_2 = test_data.iloc[:, :-27]
  test_label_2 = test_data.iloc[:, -1]
  train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
  test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

  x_train_2 = train_data_2.iloc[:, :-1]
  y_train_2 = train_data_2.iloc[:, -1]
  x_test_2 = test_data_2.iloc[:, :-1]
  y_test_2 = test_data_2.iloc[:, -1]
  xtrain2 = x_train_2.to_numpy()
  ytrain2 = y_train_2.to_numpy()
  xtest2 = x_test_2.to_numpy()
  ytest2 = y_test_2.to_numpy()

  rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_2.fit(xtrain2, ytrain2)
  ypred2 = rf_2.predict(xtest2)
  fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
  c2 = auc(fpr2, tpr2)

  #Delong Test
  p = delong_roc_test(ytest1,ypred1,ypred2)
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)
  aus2,ci2 = delong_roc_ci(ytest1,ypred2)

  c_table.loc[len(c_table)] = [c1, c2, p]

  plt.figure(figsize=(5,5))
  plt.plot(fpr1, tpr1, color='#50A0C3', lw=1, label='FGCRS+RNFL (area = %0.3f)' % c1)
  plt.plot(fpr2, tpr2, color='#F8981D', lw=1, label='FGCRS (area = %0.3f)' % c2)
  plt.plot([0, 1], [0, 1], color='#E1DCD9', lw=1, linestyle='--')
  t = ('p-value = %0.3f' % p)
  plt.text(0,1,t)
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('roc_curve_t2d_'+ str(j))
  plt.legend(loc="lower right")
  plt.savefig('roc_curve_t2d_' + str(j)+ '.png')
  plt.clf()

  arr_path = ('ypred1_t2d_'+ str(j)+ '.npy')
  np.save(arr_path, ypred1)
  arr_path = ('ypred2_t2d_'+ str(j)+ '.npy')
  np.save(arr_path, ypred2)
  arr_path = ('ytest_t2d_'+ str(j) + '.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

### Stratified Study  (Social)

In [ ]:
path3='UKB协变量_ysp20230503.dta'
convar2 = pd.read_stata(path3)
convar2 = convar2[['eid_ageing','a5']]

In [ ]:
event = event[['eid_ageing','t2d_2104']]
c_table = pd.DataFrame(columns=['RF_CB', 'RF_FGCRS', 'P-value'])

merged_df = pd.merge(metab,event, on='eid_ageing')
merged_df = pd.merge(full,merged_df, on='eid_ageing')
merged_df = pd.merge(convar2,merged_df, on='eid_ageing')
merged_df = merged_df.dropna()
df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

for j in tqdm(range(2)):
  townsend = df_shuffled.loc[df_shuffled['a5'] == j]
  townsend.drop(['a5'], axis=1, inplace=True)

  train_ratio=0.8
  test_ratio=0.2
  num_train_samples = int(len(townsend) * train_ratio)
  num_test_samples = len(townsend) - num_train_samples
  train_data = townsend[:num_train_samples].reset_index(drop=True)
  test_data = townsend[num_train_samples:].reset_index(drop=True)
  train_data.drop(['eid_ageing'], axis=1, inplace=True)
  test_data.drop(['eid_ageing'], axis=1, inplace=True)

  # Combined Model
  x_train_1 = train_data.iloc[:, :-1]
  y_train_1 = train_data.iloc[:, -1]
  x_test_1 = test_data.iloc[:, :-1]
  y_test_1 = test_data.iloc[:, -1]
  xtrain1 = x_train_1.to_numpy()
  ytrain1 = y_train_1.to_numpy()
  xtest1 = x_test_1.to_numpy()
  ytest1 = y_test_1.to_numpy()

  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)
  fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
  c1 = auc(fpr1, tpr1)

  # FGCRS
  train_data_2 = train_data.iloc[:, :-27]
  train_label_2 = train_data.iloc[:, -1]
  test_data_2 = test_data.iloc[:, :-27]
  test_label_2 = test_data.iloc[:, -1]
  train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
  test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

  x_train_2 = train_data_2.iloc[:, :-1]
  y_train_2 = train_data_2.iloc[:, -1]
  x_test_2 = test_data_2.iloc[:, :-1]
  y_test_2 = test_data_2.iloc[:, -1]
  xtrain2 = x_train_2.to_numpy()
  ytrain2 = y_train_2.to_numpy()
  xtest2 = x_test_2.to_numpy()
  ytest2 = y_test_2.to_numpy()

  rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_2.fit(xtrain2, ytrain2)
  ypred2 = rf_2.predict(xtest2)
  fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
  c2 = auc(fpr2, tpr2)

  #Delong Test
  p = delong_roc_test(ytest1,ypred1,ypred2)
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)
  aus2,ci2 = delong_roc_ci(ytest1,ypred2)

  c_table.loc[len(c_table)] = [c1, c2, p]

  arr_path = ('ypred1_t2d_'+ str(j)+ '.npy')
  np.save(arr_path, ypred1)
  arr_path = ('ypred2_t2d_'+ str(j)+ '.npy')
  np.save(arr_path, ypred2)
  arr_path = ('ytest_t2d_'+ str(j) + '.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

## Model Comparison (SCORE2)

In [ ]:
# Load data
path0='UKB协变量_ysp20230503.dta'
path1='UKB代谢组_ysp20230429.dta'
path2='UKB心血管结局_ysp20230429.dta'
path3='UKB_dmstatus.dta'
convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1','a2','a6','a16','chol_mmol','hdl']]

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1','a16','chol_mmol','hdl']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

In [ ]:
event = event[['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']]

In [ ]:
c_table = pd.DataFrame(columns=['RF_CB', 'RF_SCORE2', 'RF_RNFL','P-value'])

In [ ]:
for i in tqdm(range(event.shape[1]-1)):

  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  train_ratio=0.8
  test_ratio=0.2
  num_train_samples = int(len(df_shuffled) * train_ratio)
  num_test_samples = len(df_shuffled) - num_train_samples
  train_data = df_shuffled[:num_train_samples].reset_index(drop=True)
  test_data = df_shuffled[num_train_samples:].reset_index(drop=True)
  train_data.drop(['eid_ageing'], axis=1, inplace=True)
  test_data.drop(['eid_ageing'], axis=1, inplace=True)

  # Combined Model
  x_train_1 = train_data.iloc[:, :-1]
  y_train_1 = train_data.iloc[:, -1]
  x_test_1 = test_data.iloc[:, :-1]
  y_test_1 = test_data.iloc[:, -1]
  xtrain1 = x_train_1.to_numpy()
  ytrain1 = y_train_1.to_numpy()
  xtest1 = x_test_1.to_numpy()
  ytest1 = y_test_1.to_numpy()

  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)
  fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
  c1 = auc(fpr1, tpr1)

  # FGCRS
  train_data_2 = train_data.iloc[:, :14]
  train_label_2 = train_data.iloc[:, -1]
  test_data_2 = test_data.iloc[:, :14]
  test_label_2 = test_data.iloc[:, -1]
  train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
  test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

  x_train_2 = train_data_2.iloc[:, :-1]
  y_train_2 = train_data_2.iloc[:, -1]
  x_test_2 = test_data_2.iloc[:, :-1]
  y_test_2 = test_data_2.iloc[:, -1]
  xtrain2 = x_train_2.to_numpy()
  ytrain2 = y_train_2.to_numpy()
  xtest2 = x_test_2.to_numpy()
  ytest2 = y_test_2.to_numpy()

  rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_2.fit(xtrain2, ytrain2)
  ypred2 = rf_2.predict(xtest2)
  fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
  c2 = auc(fpr2, tpr2)

  # RNFL
  x_train_3 = train_data.iloc[:, 14:-2]
  y_train_3 = train_data.iloc[:, -1]
  x_test_3 = test_data.iloc[:, 14:-2]
  y_test_3 = test_data.iloc[:, -1]

  xtrain3 = x_train_3.to_numpy()
  ytrain3 = y_train_3.to_numpy()
  xtest3 = x_test_3.to_numpy()
  ytest3 = y_test_3.to_numpy()

  rf_3 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_3.fit(xtrain3, ytrain3)
  ypred3 = rf_3.predict(xtest3)
  fpr3, tpr3, _ = roc_curve(ytest3, ypred3)
  c3 = auc(fpr3, tpr3)

  #Delong Test
  p = delong_roc_test(ytest1,ypred1,ypred2)
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)
  aus2,ci2 = delong_roc_ci(ytest1,ypred2)

  # Results
  c_table.loc[len(c_table)] = [c1, c2, c3, p]
  col_name = disease.columns[1]


  arr_path = ('ypred1_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ypred1)
  arr_path = ('ypred2_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ypred2)
  arr_path = ('ypred3_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ypred3)
  arr_path = ('ytest_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

## Model Comparison (AHA/ASCVD)

In [ ]:
# Load data
path0='UKB协变量_ysp20230503.dta'
path1='UKB代谢组_ysp20230429.dta'
path2='UKB心血管结局_ysp20230429.dta'
path3='UKB_dmstatus.dta'
convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1','a2','a6','a8','a13','a16','chol_mmol','hdl']]
dm = dm[['eid_ageing','dmstatus']]

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1','a16','chol_mmol','hdl']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

In [ ]:
event = event[['eid_ageing','t2d_2104']] #,'mi_2104','cvddeath_status','cvd_hf_2104','cerestroke_2104','deathstatus'
c_table = pd.DataFrame(columns=['CB_CIL', 'CB_ROC', 'CB_CIH', 'CV_CIL', 'CV_ROC', 'CV_CIH', 'P'])

for i in tqdm(range(event.shape[1]-1)):

  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  #merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  train_ratio=0.8
  num_train_samples = int(len(df_shuffled) * train_ratio)
  num_test_samples = len(df_shuffled) - num_train_samples
  train_data = df_shuffled[:num_train_samples].reset_index(drop=True)
  test_data = df_shuffled[num_train_samples:].reset_index(drop=True)
  train_data.drop(['eid_ageing'], axis=1, inplace=True)
  test_data.drop(['eid_ageing'], axis=1, inplace=True)

  # Combined Model
  age_idx = train_data.columns.get_loc('a1')
  x_train_1 = train_data.iloc[:, age_idx:-1]
  y_train_1 = train_data.iloc[:, -1]
  x_test_1 = test_data.iloc[:, age_idx:-1]
  y_test_1 = test_data.iloc[:, -1]
  xtrain1 = x_train_1.to_numpy()
  ytrain1 = y_train_1.to_numpy()
  xtest1 = x_test_1.to_numpy()
  ytest1 = y_test_1.to_numpy()

  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)
  fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
  c1 = auc(fpr1, tpr1)

  # FGCRS
  metab_idx = train_data.columns.get_loc('m7')
  train_data_2 = train_data.iloc[:, age_idx:metab_idx]
  train_label_2 = train_data.iloc[:, -1]
  test_data_2 = test_data.iloc[:, age_idx:metab_idx]
  test_label_2 = test_data.iloc[:, -1]
  train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
  test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

  x_train_2 = train_data_2.iloc[:, :-1]
  y_train_2 = train_data_2.iloc[:, -1]
  x_test_2 = test_data_2.iloc[:, :-1]
  y_test_2 = test_data_2.iloc[:, -1]
  xtrain2 = x_train_2.to_numpy()
  ytrain2 = y_train_2.to_numpy()
  xtest2 = x_test_2.to_numpy()
  ytest2 = y_test_2.to_numpy()

  rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_2.fit(xtrain2, ytrain2)
  ypred2 = rf_2.predict(xtest2)
  fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
  c2 = auc(fpr2, tpr2)

  #Delong Test
  p = delong_roc_test(ytest1,ypred1,ypred2)
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)
  aus2,ci2 = delong_roc_ci(ytest1,ypred2)

  # Results
  c_table.loc[len(c_table)] = [ci1[0], aus1, ci1[1], ci2[0], aus2, ci2[1], p[0,0]]
  col_name = disease.columns[1]

  arr_path = ('ypred1_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ypred1)
  arr_path = ('ypred2_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ypred2)
  arr_path = ('ytest_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

## Model Comparison (WHO-CVD)

In [ ]:
# Load data
path0='UKB协变量_ysp20230503.dta'
path1='UKB代谢组_ysp20230429.dta'
path2='UKB心血管结局_ysp20230429.dta'
path3='UKB_dmstatus.dta'
convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1','a2','a6','a16','chol_mmol']]
dm = dm[['eid_ageing','dmstatus']]

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1','a16','chol_mmol']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

In [ ]:
event = event[['eid_ageing','t2d_2104']] #,'mi_2104','cvddeath_status','cvd_hf_2104','cerestroke_2104','deathstatus'
c_table = pd.DataFrame(columns=['CB_CIL', 'CB_ROC', 'CB_CIH', 'CV_CIL', 'CV_ROC', 'CV_CIH', 'P'])

for i in tqdm(range(event.shape[1]-1)):

  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  #merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  train_ratio=0.8
  num_train_samples = int(len(df_shuffled) * train_ratio)
  num_test_samples = len(df_shuffled) - num_train_samples
  train_data = df_shuffled[:num_train_samples].reset_index(drop=True)
  test_data = df_shuffled[num_train_samples:].reset_index(drop=True)
  train_data.drop(['eid_ageing'], axis=1, inplace=True)
  test_data.drop(['eid_ageing'], axis=1, inplace=True)

  # Combined Model
  age_idx = train_data.columns.get_loc('a1')
  x_train_1 = train_data.iloc[:, age_idx:-1]
  y_train_1 = train_data.iloc[:, -1]
  x_test_1 = test_data.iloc[:, age_idx:-1]
  y_test_1 = test_data.iloc[:, -1]
  xtrain1 = x_train_1.to_numpy()
  ytrain1 = y_train_1.to_numpy()
  xtest1 = x_test_1.to_numpy()
  ytest1 = y_test_1.to_numpy()

  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)
  fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
  c1 = auc(fpr1, tpr1)

  # FGCRS
  metab_idx = train_data.columns.get_loc('m7')
  train_data_2 = train_data.iloc[:, age_idx:metab_idx]
  train_label_2 = train_data.iloc[:, -1]
  test_data_2 = test_data.iloc[:, age_idx:metab_idx]
  test_label_2 = test_data.iloc[:, -1]
  train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
  test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

  x_train_2 = train_data_2.iloc[:, :-1]
  y_train_2 = train_data_2.iloc[:, -1]
  x_test_2 = test_data_2.iloc[:, :-1]
  y_test_2 = test_data_2.iloc[:, -1]
  xtrain2 = x_train_2.to_numpy()
  ytrain2 = y_train_2.to_numpy()
  xtest2 = x_test_2.to_numpy()
  ytest2 = y_test_2.to_numpy()

  rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_2.fit(xtrain2, ytrain2)
  ypred2 = rf_2.predict(xtest2)
  fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
  c2 = auc(fpr2, tpr2)

  #Delong Test
  p = delong_roc_test(ytest1,ypred1,ypred2)
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)
  aus2,ci2 = delong_roc_ci(ytest1,ypred2)

  # Results
  c_table.loc[len(c_table)] = [ci1[0], aus1, ci1[1], ci2[0], aus2, ci2[1], p[0,0]]
  col_name = disease.columns[1]


  arr_path = ('ypred1_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ypred1)
  arr_path = ('ypred2_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ypred2)
  arr_path = ('ytest_'+f"{col_name}_" +'.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

## Stratified Study (Sex)

In [ ]:
# Load data
path0='UKB协变量_ysp20230503.dta'
path1='UKB代谢组_ysp20230429.dta'
path2='UKB心血管结局_ysp20230429.dta'
path3='UKB_dmstatus.dta'
convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1','a2','a6','a16','a13','chol_mmol','hdl']]
dm = dm[['eid_ageing','dmstatus']]

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1','a16','chol_mmol','hdl']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

In [ ]:
event = event[['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']]

Population

In [ ]:
disease = ['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']
population = pd.DataFrame(columns=['female','female_disease','female_ratio','male','male_disease','male_ratio'])

for i in tqdm(range(event.shape[1]-1)):
  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  # balance edu dataset
  nonuniv = df_shuffled.loc[df_shuffled['a2'] == 0]
  univ = df_shuffled.loc[df_shuffled['a2'] == 1]
  nonuniv_r = nonuniv[nonuniv.iloc[:, -1] == 1]
  univ_r = univ[univ.iloc[:, -1] == 1]

  x1 = len(nonuniv)
  x2 = len(univ)
  x3 = len(nonuniv_r)
  x4 = len(univ_r)
  r1 = x3/x1
  r2 = x4/x2

  population.loc[len(population)] = [x1,x3,r1,x2,x4,r2]

population.to_csv('population.csv', index=False)

Prediction

In [ ]:
c_table = pd.DataFrame(columns=['RF_CB', 'RF_FGCRS', 'P-value'])

In [ ]:
for i in tqdm(range(event.shape[1]-1)):

  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  # balance sex dataset
  female = len(df_shuffled.loc[df_shuffled['a2'] == 0])
  male = len(df_shuffled.loc[df_shuffled['a2'] == 1])

  for j in range(2):
    sex = df_shuffled.loc[df_shuffled['a2'] == j]
    sex.drop(['a2'], axis=1, inplace=True)

    train_ratio=0.8
    test_ratio=0.2
    num_train_samples = int(len(sex) * train_ratio)
    num_test_samples = len(sex) - num_train_samples
    train_data = sex[:num_train_samples].reset_index(drop=True)
    test_data = sex[num_train_samples:].reset_index(drop=True)
    train_data.drop(['eid_ageing'], axis=1, inplace=True)
    test_data.drop(['eid_ageing'], axis=1, inplace=True)

    if male > female and j==0:
      indexlist = list(range(1,int(female*0.8)))
      random.seed(100)
      oversample = np.array(random.sample(indexlist, k=int((male-female)*0.8)))
      aug = train_data.iloc[oversample, :]
      train_data = pd.concat([train_data, aug], axis=0)

    if female >= male and j==1:
      indexlist = list(range(1,int(male*0.8)))
      random.seed(100)
      oversample = np.array(random.sample(indexlist, k=int((female-male)*0.8)))
      aug = train_data.iloc[oversample, :]
      train_data = pd.concat([train_data, aug], axis=0)


    # Combined Model
    x_train_1 = train_data.iloc[:, :-1]
    y_train_1 = train_data.iloc[:, -1]
    x_test_1 = test_data.iloc[:, :-1]
    y_test_1 = test_data.iloc[:, -1]
    xtrain1 = x_train_1.to_numpy()
    ytrain1 = y_train_1.to_numpy()
    xtest1 = x_test_1.to_numpy()
    ytest1 = y_test_1.to_numpy()

    rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_1.fit(xtrain1, ytrain1)
    ypred1 = rf_1.predict(xtest1)
    fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
    c1 = auc(fpr1, tpr1)

    # FGCRS
    train_data_2 = train_data.iloc[:, :15]
    train_label_2 = train_data.iloc[:, -1]
    test_data_2 = test_data.iloc[:, :15]
    test_label_2 = test_data.iloc[:, -1]
    train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
    test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

    x_train_2 = train_data_2.iloc[:, :-1]
    y_train_2 = train_data_2.iloc[:, -1]
    x_test_2 = test_data_2.iloc[:, :-1]
    y_test_2 = test_data_2.iloc[:, -1]
    xtrain2 = x_train_2.to_numpy()
    ytrain2 = y_train_2.to_numpy()
    xtest2 = x_test_2.to_numpy()
    ytest2 = y_test_2.to_numpy()

    rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_2.fit(xtrain2, ytrain2)
    ypred2 = rf_2.predict(xtest2)
    fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
    c2 = auc(fpr2, tpr2)

    #Delong Test
    p = delong_roc_test(ytest1,ypred1,ypred2)
    aus1,ci1 = delong_roc_ci(ytest1,ypred1)
    aus2,ci2 = delong_roc_ci(ytest1,ypred2)

    # Results
    c_table.loc[len(c_table)] = [c1, c2, p]
    col_name = disease.columns[1]


    arr_path = ('ypred1_'+f"{col_name}_" + str(j) +'.npy')
    np.save(arr_path, ypred1)
    arr_path = ('ypred2_'+f"{col_name}_" + str(j)+'.npy')
    np.save(arr_path, ypred2)
    arr_path = ('ytest_'+f"{col_name}_" + str(j)+'.npy')
    np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

## Stratified Study (Education)

In [ ]:
# Load data
path0='UKB协变量_ysp20230503.dta'
path1='UKB代谢组_ysp20230429.dta'
path2='UKB心血管结局_ysp20230429.dta'
path3='UKB_dmstatus.dta'
convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1','a2','a9','a6','a16','a13','chol_mmol','hdl']] #education index
dm = dm[['eid_ageing','dmstatus']]

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1','a16','chol_mmol','hdl']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

In [ ]:
event = event[['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']]

population

In [ ]:
disease = ['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']
population = pd.DataFrame(columns=['nonuniv','nonuniv_disease','nonuniv_ratio','univ','univ_disease','univ_ratio'])

for i in tqdm(range(event.shape[1]-1)):
  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  # balance edu dataset
  nonuniv = df_shuffled.loc[df_shuffled['a9'] == 0]
  univ = df_shuffled.loc[df_shuffled['a9'] == 1]
  nonuniv_r = nonuniv[nonuniv.iloc[:, -1] == 1]
  univ_r = univ[univ.iloc[:, -1] == 1]

  x1 = len(nonuniv)
  x2 = len(univ)
  x3 = len(nonuniv_r)
  x4 = len(univ_r)
  r1 = x3/x1
  r2 = x4/x2

  population.loc[len(population)] = [x1,x3,r1,x2,x4,r2]


In [ ]:
c_table = pd.DataFrame(columns=['CB_CIL', 'CB_ROC', 'CB_CIH', 'CV_CIL', 'CV_ROC', 'CV_CIH', 'P'])

for i in tqdm(range(event.shape[1]-1)):

  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  # balance edu dataset
  nonuniv = len(df_shuffled.loc[df_shuffled['a9'] == 0])
  univ = len(df_shuffled.loc[df_shuffled['a9'] == 1])

  for j in range(2):
    edu = df_shuffled.loc[df_shuffled['a9'] == j]
    edu.drop(['a9'], axis=1, inplace=True)

    # undersample
    if univ > nonuniv and j==1:
      edu = edu[:nonuniv].reset_index(drop=True)

    if nonuniv >= univ and j==0:
      edu = edu[:univ].reset_index(drop=True)

    train_ratio=0.8
    num_train_samples = int(len(edu) * train_ratio)
    num_test_samples = len(edu) - num_train_samples
    train_data = edu[:num_train_samples].reset_index(drop=True)
    test_data = edu[num_train_samples:].reset_index(drop=True)
    train_data.drop(['eid_ageing'], axis=1, inplace=True)
    test_data.drop(['eid_ageing'], axis=1, inplace=True)

    # Combined Model
    age_idx = train_data.columns.get_loc('a1')
    x_train_1 = train_data.iloc[:, age_idx:-1]
    y_train_1 = train_data.iloc[:, -1]
    x_test_1 = test_data.iloc[:, age_idx:-1]
    y_test_1 = test_data.iloc[:, -1]
    xtrain1 = x_train_1.to_numpy()
    ytrain1 = y_train_1.to_numpy()
    xtest1 = x_test_1.to_numpy()
    ytest1 = y_test_1.to_numpy()

    rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_1.fit(xtrain1, ytrain1)
    ypred1 = rf_1.predict(xtest1)
    fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
    c1 = auc(fpr1, tpr1)

    # FGCRS
    metab_idx = train_data.columns.get_loc('m7')
    train_data_2 = train_data.iloc[:, age_idx:metab_idx]
    train_label_2 = train_data.iloc[:, -1]
    test_data_2 = test_data.iloc[:, age_idx:metab_idx]
    test_label_2 = test_data.iloc[:, -1]
    train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
    test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

    x_train_2 = train_data_2.iloc[:, :-1]
    y_train_2 = train_data_2.iloc[:, -1]
    x_test_2 = test_data_2.iloc[:, :-1]
    y_test_2 = test_data_2.iloc[:, -1]
    xtrain2 = x_train_2.to_numpy()
    ytrain2 = y_train_2.to_numpy()
    xtest2 = x_test_2.to_numpy()
    ytest2 = y_test_2.to_numpy()

    rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_2.fit(xtrain2, ytrain2)
    ypred2 = rf_2.predict(xtest2)
    fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
    c2 = auc(fpr2, tpr2)

    #Delong Test
    p = delong_roc_test(ytest1,ypred1,ypred2)
    aus1,ci1 = delong_roc_ci(ytest1,ypred1)
    aus2,ci2 = delong_roc_ci(ytest1,ypred2)

    # Results
    c_table.loc[len(c_table)] = [ci1[0], aus1, ci1[1], ci2[0], aus2, ci2[1], p[0,0]]

    col_name = disease.columns[1]

    arr_path = ('ypred1_'+f"{col_name}_" + str(j) +'.npy')
    np.save(arr_path, ypred1)
    arr_path = ('ypred2_'+f"{col_name}_" + str(j)+'.npy')
    np.save(arr_path, ypred2)
    arr_path = ('ytest_'+f"{col_name}_" + str(j)+'.npy')
    np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

## Stratified Study (Social)

In [ ]:
# Load data
path0='UKB协变量_ysp20230503.dta'
path1='UKB代谢组_ysp20230429.dta'
path2='UKB心血管结局_ysp20230429.dta'
path3='UKB_dmstatus.dta'
convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1','a2','a5','a6','a16','a13','chol_mmol','hdl']] #a5 townsend depreviation index
dm = dm[['eid_ageing','dmstatus']]

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1','a16','chol_mmol','hdl']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

Population

In [ ]:
event = event[['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']]

In [ ]:
disease = ['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']
population = pd.DataFrame(columns=['well','well_disease','well_ratio','poor','poor_disease','poor_ratio'])

for i in tqdm(range(event.shape[1]-1)):
  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  # balance edu dataset
  nonuniv = df_shuffled.loc[df_shuffled['a5'] == 0]
  univ = df_shuffled.loc[df_shuffled['a5'] == 1]
  nonuniv_r = nonuniv[nonuniv.iloc[:, -1] == 1]
  univ_r = univ[univ.iloc[:, -1] == 1]

  x1 = len(nonuniv)
  x2 = len(univ)
  x3 = len(nonuniv_r)
  x4 = len(univ_r)
  r1 = x3/x1
  r2 = x4/x2

  population.loc[len(population)] = [x1,x3,r1,x2,x4,r2]

population.to_csv('population.csv', index=False)

Prediction

In [ ]:
event = event[['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']]
c_table = pd.DataFrame(columns=['RF_CB', 'RF_FGCRS','P-value'])

for i in tqdm(range(event.shape[1]-1)):

  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  for j in range(2):
    townsend = df_shuffled.loc[df_shuffled['a5'] == j]
    townsend.drop(['a5'], axis=1, inplace=True)

    train_ratio=0.8
    test_ratio=0.2
    num_train_samples = int(len(townsend) * train_ratio)
    num_test_samples = len(townsend) - num_train_samples
    train_data = townsend[:num_train_samples].reset_index(drop=True)
    test_data = townsend[num_train_samples:].reset_index(drop=True)
    train_data.drop(['eid_ageing'], axis=1, inplace=True)
    test_data.drop(['eid_ageing'], axis=1, inplace=True)

    # Combined Model
    x_train_1 = train_data.iloc[:, :-1]
    y_train_1 = train_data.iloc[:, -1]
    x_test_1 = test_data.iloc[:, :-1]
    y_test_1 = test_data.iloc[:, -1]
    xtrain1 = x_train_1.to_numpy()
    ytrain1 = y_train_1.to_numpy()
    xtest1 = x_test_1.to_numpy()
    ytest1 = y_test_1.to_numpy()

    rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_1.fit(xtrain1, ytrain1)
    ypred1 = rf_1.predict(xtest1)
    fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
    c1 = auc(fpr1, tpr1)

    # FGCRS
    train_data_2 = train_data.iloc[:, :16]
    train_label_2 = train_data.iloc[:, -1]
    test_data_2 = test_data.iloc[:, :16]
    test_label_2 = test_data.iloc[:, -1]
    train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
    test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

    x_train_2 = train_data_2.iloc[:, :-1]
    y_train_2 = train_data_2.iloc[:, -1]
    x_test_2 = test_data_2.iloc[:, :-1]
    y_test_2 = test_data_2.iloc[:, -1]
    xtrain2 = x_train_2.to_numpy()
    ytrain2 = y_train_2.to_numpy()
    xtest2 = x_test_2.to_numpy()
    ytest2 = y_test_2.to_numpy()

    rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_2.fit(xtrain2, ytrain2)
    ypred2 = rf_2.predict(xtest2)
    fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
    c2 = auc(fpr2, tpr2)

    #Delong Test
    p = delong_roc_test(ytest1,ypred1,ypred2)
    aus1,ci1 = delong_roc_ci(ytest1,ypred1)
    aus2,ci2 = delong_roc_ci(ytest1,ypred2)

    # Results
    c_table.loc[len(c_table)] = [c1, c2, p]
    col_name = disease.columns[1]


    arr_path = ('ypred1_'+f"{col_name}_" + str(j) +'.npy')
    np.save(arr_path, ypred1)
    arr_path = ('ypred2_'+f"{col_name}_" + str(j)+'.npy')
    np.save(arr_path, ypred2)
    arr_path = ('ytest_'+f"{col_name}_" + str(j)+'.npy')
    np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table.csv', index=False)

## Stratified Study (Genes)

In [ ]:
# Load data
path0='UKB协变量_ysp20230503.dta'
path1='UKB代谢组_ysp20230429.dta'
path2='UKB心血管结局_ysp20230429.dta'
path3='UKB_dmstatus.dta'
path4='UKB多基因评分_ysp20230818.dta'
convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)
genes=pd.read_stata(path4)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1','a2','a6','a16','a13','chol_mmol','hdl']] #education index
dm = dm[['eid_ageing','dmstatus']]
genes = genes[['eid_ageing','n_26285_0_0']]
convar = pd.merge(convar,genes, on='eid_ageing')

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1','a16','chol_mmol','hdl']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

cutoff_percentage = 0.5
convar = convar.dropna(subset=['n_26285_0_0'])
sorted_df = convar.sort_values(by='n_26285_0_0')

total_count = len(sorted_df)
cutoff_index = int(total_count * cutoff_percentage)
sorted_df.loc[sorted_df.index[:cutoff_index], 'n_26285_0_0'] = 0
sorted_df.loc[sorted_df.index[cutoff_index:], 'n_26285_0_0'] = 1
result_df = sorted_df.sort_index()

In [ ]:
event = event[['eid_ageing','t2d_2104']] #,'mi_2104','cvd_hf_2104','cerestroke_2104','deathstatus','cvddeath_status'
c_table = pd.DataFrame(columns=['CB_CIL', 'CB_ROC', 'CB_CIH', 'CV_CIL', 'CV_ROC', 'CV_CIH', 'P'])

for i in tqdm(range(event.shape[1]-1)):

  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(result_df,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  for j in range(2):
    gene = df_shuffled.loc[df_shuffled['n_26285_0_0'] == j]
    gene.drop(['n_26285_0_0'], axis=1, inplace=True)

    train_ratio=0.8
    num_train_samples = int(len(gene) * train_ratio)
    train_data = gene[:num_train_samples].reset_index(drop=True)
    test_data = gene[num_train_samples:].reset_index(drop=True)
    train_data.drop(['eid_ageing'], axis=1, inplace=True)
    test_data.drop(['eid_ageing'], axis=1, inplace=True)

    # Combined Model
    age_idx = train_data.columns.get_loc('a1')
    x_train_1 = train_data.iloc[:, age_idx:-1]
    y_train_1 = train_data.iloc[:, -1]
    x_test_1 = test_data.iloc[:, age_idx:-1]
    y_test_1 = test_data.iloc[:, -1]
    xtrain1 = x_train_1.to_numpy()
    ytrain1 = y_train_1.to_numpy()
    xtest1 = x_test_1.to_numpy()
    ytest1 = y_test_1.to_numpy()

    rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_1.fit(xtrain1, ytrain1)
    ypred1 = rf_1.predict(xtest1)

    # FGCRS
    metab_idx = train_data.columns.get_loc('m7')
    train_data_2 = train_data.iloc[:, age_idx:metab_idx]
    train_label_2 = train_data.iloc[:, -1]
    test_data_2 = test_data.iloc[:, age_idx:metab_idx]
    test_label_2 = test_data.iloc[:, -1]
    train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
    test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

    x_train_2 = train_data_2.iloc[:, :-1]
    y_train_2 = train_data_2.iloc[:, -1]
    x_test_2 = test_data_2.iloc[:, :-1]
    y_test_2 = test_data_2.iloc[:, -1]
    xtrain2 = x_train_2.to_numpy()
    ytrain2 = y_train_2.to_numpy()
    xtest2 = x_test_2.to_numpy()
    ytest2 = y_test_2.to_numpy()

    rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_2.fit(xtrain2, ytrain2)
    ypred2 = rf_2.predict(xtest2)

    #Delong Test
    p = delong_roc_test(ytest1,ypred1,ypred2)
    aus1,ci1 = delong_roc_ci(ytest1,ypred1)
    aus2,ci2 = delong_roc_ci(ytest1,ypred2)

    # Results
    c_table.loc[len(c_table)] = [ci1[0], aus1, ci1[1], ci2[0], aus2, ci2[1], p[0,0]]

    col_name = disease.columns[1]

    arr_path = ('ypred1_'+f"{col_name}_" + str(j) +'.npy')
    np.save(arr_path, ypred1)
    arr_path = ('ypred2_'+f"{col_name}_" + str(j)+'.npy')
    np.save(arr_path, ypred2)
    arr_path = ('ytest_'+f"{col_name}_" + str(j)+'.npy')
    np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('c_table_t2d.csv', index=False)

## Stratified Study (Income)

In [ ]:
# Load data
path0='UKB协变量_ysp20230503.dta'
path1='UKB代谢组_ysp20230429.dta'
path2='UKB心血管结局_ysp20230429.dta'
path3='UKB_dmstatus.dta'
convar=pd.read_stata(path0)
metab=pd.read_stata(path1)
event=pd.read_stata(path2)
dm=pd.read_stata(path3)

metab = metab[['eid_ageing','m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']]
convar = convar[['eid_ageing','a1','a2','a4','a6','a16','a13','chol_mmol','hdl']] # a4: average total household income before tax
dm = dm[['eid_ageing','dmstatus']]

In [ ]:
# normalization
scaler = MinMaxScaler()
columns_to_normalize=['a1','a16','chol_mmol','hdl']
convar[columns_to_normalize] = scaler.fit_transform(convar[columns_to_normalize])
columns_to_normalize = ['m7','m12','m15','m19','m23','m27','m28','m31','m35','m36','m37','m40','m154','m155','m156','m157','m158','m159','m161','m162','m163','m164','m165','m166','m205','m210']
metab[columns_to_normalize] = scaler.fit_transform(metab[columns_to_normalize])

In [ ]:
event = event[['eid_ageing','deathstatus','cvd_hf_2104','cerestroke_2104','t2d_2104','mi_2104','cvddeath_status']]

In [ ]:
c_table = pd.DataFrame(columns=['RF_CB', 'RF_FGCRS','P-value'])

In [ ]:
for i in tqdm(range(event.shape[1]-1)):

  disease =  event.iloc[:, [0,i+1]]
  merged_df = pd.merge(metab,disease, on='eid_ageing')
  merged_df = pd.merge(dm,merged_df, on='eid_ageing')
  merged_df = pd.merge(convar,merged_df, on='eid_ageing')
  merged_df = merged_df.dropna()
  df_shuffled = merged_df.sample(frac=1,random_state=100).reset_index(drop=True)

  for j in range(2):
    townsend = df_shuffled.loc[df_shuffled['a4'] == j]
    townsend.drop(['a4'], axis=1, inplace=True)

    train_ratio=0.8
    test_ratio=0.2
    num_train_samples = int(len(townsend) * train_ratio)
    num_test_samples = len(townsend) - num_train_samples
    train_data = townsend[:num_train_samples].reset_index(drop=True)
    test_data = townsend[num_train_samples:].reset_index(drop=True)
    train_data.drop(['eid_ageing'], axis=1, inplace=True)
    test_data.drop(['eid_ageing'], axis=1, inplace=True)

    # Combined Model
    x_train_1 = train_data.iloc[:, :-1]
    y_train_1 = train_data.iloc[:, -1]
    x_test_1 = test_data.iloc[:, :-1]
    y_test_1 = test_data.iloc[:, -1]
    xtrain1 = x_train_1.to_numpy()
    ytrain1 = y_train_1.to_numpy()
    xtest1 = x_test_1.to_numpy()
    ytest1 = y_test_1.to_numpy()

    rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_1.fit(xtrain1, ytrain1)
    ypred1 = rf_1.predict(xtest1)
    fpr1, tpr1, _ = roc_curve(ytest1, ypred1)
    c1 = auc(fpr1, tpr1)

    # FGCRS
    train_data_2 = train_data.iloc[:, :16]
    train_label_2 = train_data.iloc[:, -1]
    test_data_2 = test_data.iloc[:, :16]
    test_label_2 = test_data.iloc[:, -1]
    train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
    test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

    x_train_2 = train_data_2.iloc[:, :-1]
    y_train_2 = train_data_2.iloc[:, -1]
    x_test_2 = test_data_2.iloc[:, :-1]
    y_test_2 = test_data_2.iloc[:, -1]
    xtrain2 = x_train_2.to_numpy()
    ytrain2 = y_train_2.to_numpy()
    xtest2 = x_test_2.to_numpy()
    ytest2 = y_test_2.to_numpy()

    rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
    rf_2.fit(xtrain2, ytrain2)
    ypred2 = rf_2.predict(xtest2)
    fpr2, tpr2, _ = roc_curve(ytest2, ypred2)
    c2 = auc(fpr2, tpr2)

    #Delong Test
    p = delong_roc_test(ytest1,ypred1,ypred2)
    aus1,ci1 = delong_roc_ci(ytest1,ypred1)
    aus2,ci2 = delong_roc_ci(ytest1,ypred2)

    # Results
    c_table.loc[len(c_table)] = [c1, c2, p]
    col_name = disease.columns[1]


    arr_path = ('ypred1_'+f"{col_name}_" + str(j) +'.npy')
    np.save(arr_path, ypred1)
    arr_path = ('ypred2_'+f"{col_name}_" + str(j)+'.npy')
    np.save(arr_path, ypred2)
    arr_path = ('ytest_'+f"{col_name}_" + str(j)+'.npy')
    np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('syndata/c_table.csv', index=False)

## Synthetic data analysis

### Comparison of individual predictors

In [ ]:
path = 'data/Sample Data.csv'
synthetic_data = pd.read_csv(path)

In [ ]:
event = 'cvd'

convar=['age', 'sex', 'smoker', 'dmduration', 'y0_sbp','y0_dbp','y0_hba1c','y0_chol','y0_hdlc', 'egfr_y0', 'med_insulin','bmi','rnfl']

c_table = pd.DataFrame(columns=['CIL', 'ROC', 'CIH'])

for i in tqdm(range(len(convar))):
  df_shuffled = synthetic_data.sample(frac=1,random_state=100).reset_index(drop=True)

  train_ratio=0.8
  num_train_samples = int(len(df_shuffled) * train_ratio)
  num_test_samples = len(df_shuffled) - num_train_samples
  train_data = df_shuffled[:num_train_samples].reset_index(drop=True)
  test_data = df_shuffled[num_train_samples:].reset_index(drop=True)

  if i<len(convar)-1:
      idx = train_data.columns.get_loc(convar[i])
      x_train_1 = train_data.iloc[:, idx]
      y_train_1 = train_data.iloc[:, -1]
      x_test_1 = test_data.iloc[:, idx]
      y_test_1 = test_data.iloc[:, -1]
      xtrain1 = x_train_1.to_numpy()
      ytrain1 = y_train_1.to_numpy()
      xtest1 = x_test_1.to_numpy()
      ytest1 = y_test_1.to_numpy()

      xtrain1=xtrain1.reshape(-1, 1)
      xtest1=xtest1.reshape(-1, 1)

  else:
      idx = train_data.columns.get_loc('m1')
      x_train_1 = train_data.iloc[:, idx:-1]
      y_train_1 = train_data.iloc[:, -1]
      x_test_1 = test_data.iloc[:, idx:-1]
      y_test_1 = test_data.iloc[:, -1]
      xtrain1 = x_train_1.to_numpy()
      ytrain1 = y_train_1.to_numpy()
      xtest1 = x_test_1.to_numpy()
      ytest1 = y_test_1.to_numpy()


  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)

  # Results
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)

  c_table.loc[len(c_table)] = [ci1[0], aus1, ci1[1]]

  arr_path = ('syndata/pred/ypred1_'+f'{convar[i]}'+'.npy')
  np.save(arr_path, ypred1)

  arr_path = ('syndata/pred/ytest_'+f'{convar[i]}'+'.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('syndata/singleconv_roctable.csv', index=False)

### Comparison of model performance

In [ ]:
def prepare_model_data(synthetic_data, model_name):

    if model_name == 'AgeSex':
        #AgeSex model
        conv_cols = ['age', 'sex']
    elif model_name == 'FGCRS':
        #FGCRS model
        conv_cols = ['age', 'sex', 'smoker', 'y0_sbp', 'med_hpt', 'y0_chol', 'y0_hdlc', 'dmduration']
    elif model_name == 'UKPDS':
        #UKPDS model
        conv_cols = ['age', 'sex', 'smoker', 'y0_hba1c', 'y0_sbp', 'y0_chol', 'y0_hdlc']
    elif model_name == 'Wan':
        #Wan model
        conv_cols = ['age', 'sex', 'dmduration', 'bmi', 'y0_sbp', 'y0_dbp', 'y0_hba1c',
                     'y0_chol', 'y0_hdlc', 'egfr_y0', 'med_hpt', 'med_insulin']
    elif model_name == 'DCS':
        #DCS model
        conv_cols = ['age', 'sex', 'dmduration', 'smoker', 'y0_sbp', 'y0_hba1c',
                     'y0_chol', 'y0_hdlc']
    else:
        raise ValueError(f"Unknown model name: {model_name}")

    convar = synthetic_data[conv_cols].copy()

    metab_idx = synthetic_data.columns.get_loc('m1')
    metab_cols = synthetic_data.iloc[:, metab_idx:]

    merged_data = pd.concat([convar, metab_cols], axis=1)

    return merged_data



In [ ]:
models = ['AgeSex', 'FGCRS', 'UKPDS', 'Wan', 'DCS']
c_table = pd.DataFrame(columns=['CB_CIL', 'CB_ROC', 'CB_CIH', 'CV_CIL', 'CV_ROC', 'CV_CIH', 'P'])

for i in tqdm(range(len(models))):
  model = models[i]
  merged_data = prepare_model_data(synthetic_data, model)
  df_shuffled = merged_data.sample(frac=1,random_state=100).reset_index(drop=True)

  train_ratio=0.8
  num_train_samples = int(len(df_shuffled) * train_ratio)
  num_test_samples = len(df_shuffled) - num_train_samples
  train_data = df_shuffled[:num_train_samples].reset_index(drop=True)
  test_data = df_shuffled[num_train_samples:].reset_index(drop=True)

  # Combined Model
  age_idx = train_data.columns.get_loc('age')
  x_train_1 = train_data.iloc[:, age_idx:-1]
  y_train_1 = train_data.iloc[:, -1]
  x_test_1 = test_data.iloc[:, age_idx:-1]
  y_test_1 = test_data.iloc[:, -1]
  xtrain1 = x_train_1.to_numpy()
  ytrain1 = y_train_1.to_numpy()
  xtest1 = x_test_1.to_numpy()
  ytest1 = y_test_1.to_numpy()

  rf_1 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_1.fit(xtrain1, ytrain1)
  ypred1 = rf_1.predict(xtest1)

  # clinical model
  metab_idx = train_data.columns.get_loc('m1')
  train_data_2 = train_data.iloc[:, age_idx:metab_idx]
  train_label_2 = train_data.iloc[:, -1]
  test_data_2 = test_data.iloc[:, age_idx:metab_idx]
  test_label_2 = test_data.iloc[:, -1]
  train_data_2 = pd.concat([train_data_2, train_label_2], axis=1)
  test_data_2 = pd.concat([test_data_2, test_label_2], axis=1)

  x_train_2 = train_data_2.iloc[:, :-1]
  y_train_2 = train_data_2.iloc[:, -1]
  x_test_2 = test_data_2.iloc[:, :-1]
  y_test_2 = test_data_2.iloc[:, -1]
  xtrain2 = x_train_2.to_numpy()
  ytrain2 = y_train_2.to_numpy()
  xtest2 = x_test_2.to_numpy()
  ytest2 = y_test_2.to_numpy()

  rf_2 = RandomForestRegressor(n_estimators=200, max_depth=10, oob_score=True, random_state = 100)
  rf_2.fit(xtrain2, ytrain2)
  ypred2 = rf_2.predict(xtest2)

  # Results
  p = delong_roc_test(ytest1,ypred1,ypred2)
  aus1,ci1 = delong_roc_ci(ytest1,ypred1)
  aus2,ci2 = delong_roc_ci(ytest2,ypred2)

  c_table.loc[len(c_table)] = [ci1[0], aus1, ci1[1], ci2[0], aus2, ci2[1], p[0,0]]

  arr_path = ('syndata/models/'+f'{model}'+'_ypred1.npy')
  np.save(arr_path, ypred1)
  arr_path = ('syndata/models/'+f'{model}'+'_ypred2.npy')
  np.save(arr_path, ypred2)
  arr_path = ('syndata/models/'+f'{model}'+'_ytest.npy')
  np.save(arr_path, ytest1)

# Save table file
c_table.to_csv('syndata/models_roctable.csv', index=False)